Пример кода с парсингом книг из определенной категории


In [54]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from pprint import pprint

In [55]:
# сделаем рандомный UserAgent
ua = UserAgent()
ua.random

# для примера взял жанр Nonfiction
url = 'https://books.toscrape.com/catalogue/category/books/nonfiction_13/'
index = 'page-1.html'

headers = {
    "User-Agent": ua.random} 

# запускаем сессию
session = requests.Session()

In [56]:
# задаем пустой список всех книг
all_book = []
# номеруем страницы (чтобы получать отчет о работе)
page = 1

# запускаем цикл сбора данных
while True:
    # делаем запрос и сохраняем ответ
    response = session.get(url+index, headers=headers)
    # проверяем код ответа
    if response.status_code < 400:
        soup_page = BeautifulSoup(response.text, 'html.parser')
    
    # ищем все книги на странице
    books_list = soup_page.find_all('article', ('class', 'product_pod'))
    
    # перебираем каждую книгу на странице
    for i in books_list:
        book = {}
        # для записи книги берем ее название и цену
        book['name'] = i.find('h3').find('a').get('title')
        book['price'] = float(i.find('div', { 'class': 'product_price'}).find('p', { 'class': 'price_color'}).getText()[2:])
        rating = i.find('p', class_='star-rating')['class'][1]
        book['rating'] = rating

        url_book = url+i.find('h3').find('a').get('href')
        res = session.get(url_book)
        qua = BeautifulSoup(res.text, 'html.parser').find('th', text='Availability').find_next_sibling('td')
        book['quantity'] = int(qua.text.split()[2][1:])
        des_l = BeautifulSoup(res.text, 'html.parser').find_all('p')
        des = des_l[3].getText()
        book['description'] = des[:150]+'...' if len(des) > 150 else des
        product_code = BeautifulSoup(res.text, 'html.parser').find('th', text='UPC').find_next_sibling('td')
        book['UPC'] = product_code.text
        all_book.append(book)

    # отчет о проделаной работе
    print(f'Книги с {page} страницы собраны')
    page += 1


    # проверяем есть ли кпонка перехода на следующую страницу
    try:   
        # если кнопка есть, то присваиваем ее значению в переменную index и искользуем дальше
        index = soup_page.find('li', { 'class': 'next'}).find('a').get('href')
        # отчет о проделанной работе
        print('Кнопка есть ')
    except:
        # если кнопки нет, выводим сообщение и завершаем цикл сбора книг
        print('EXIT')
        break
    

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3144\2033136144.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  qua = BeautifulSoup(res.text, 'html.parser').find('th', text='Availability').find_next_sibling('td')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3144\2033136144.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  product_code = BeautifulSoup(res.text, 'html.parser').find('th', text='UPC').find_next_sibling('td')


Книги с 1 страницы собраны
Кнопка есть 
Книги с 2 страницы собраны
Кнопка есть 
Книги с 3 страницы собраны
Кнопка есть 
Книги с 4 страницы собраны
Кнопка есть 
Книги с 5 страницы собраны
Кнопка есть 
Книги с 6 страницы собраны
EXIT


In [57]:
# выводим список книг этой категории
pprint(all_book)

[{'UPC': '4c28def39d850cdf',
  'description': 'Anti-apartheid activist, Bollywood screenwriter, Nazi '
                 'pin-up, hero of the Wild West: this is Shakespeare as you '
                 'have never seen him before.From the si...',
  'name': 'Worlds Elsewhere: Journeys Around Shakespeareâ\x80\x99s Globe',
  'price': 40.3,
  'quantity': 18,
  'rating': 'Five'},
 {'UPC': '3c039985229453bf',
  'description': 'Are you and your spouse speaking the same language? While '
                 'love is a many splendored thing, it is sometimes a very '
                 'confusing thing, too. And as people ...',
  'name': 'The Five Love Languages: How to Express Heartfelt Commitment to '
          'Your Mate',
  'price': 31.05,
  'quantity': 18,
  'rating': 'Three'},
 {'UPC': 'c8f7f0cb1abb9cac',
  'description': 'I want life. I want to read it and write it and feel it and '
                 'live it. I want, for as much of the time as possible in this '
                 'blink-of-an-eye e

Вывод: в результате работы получили список книг, который состоит из набора словарей, характеризующих каждую книгу с ключами "название" и "цена"

UPD:
Добавил в получаемую таблицу рейтинг книги, кол-во товара на склад,краткое описание (до 150 слов) и UPC (уникальный код продукта)